In [1]:
import lzma
import json
import pandas as pd
import numpy as np
import re
import glob
import os
import sys
import multiprocessing as mp
import pickle

In [2]:
os.chdir("data\loc_data")

In [3]:
os.getcwd()

'c:\\Users\\erica\\OneDrive - University of Southern California\\Projects_at_USC\\CKIDS_current\\data\\loc_data'

In [4]:
# lists of file names
# json xz
json_files = glob.glob("*_UTC.json")
# descrip txts
txt_files = glob.glob("*_UTC.txt")
# location txts
loc_files = glob.glob("*_location.txt")
# comments jsons
cmt_files = glob.glob("*_comments.json")


print(len(json_files))
print(len(txt_files))
print(len(loc_files))
print(len(cmt_files))

1248
1235
1249
795


In [6]:
# with open('jsons.pkl', 'rb') as f:
#     jsons = pickle.load(f)

# print(len(jsons))

1248


In [33]:
# Add all json files to a dataframe
temp = pd.DataFrame()

path_to_json = 'data\loc_data' 

json_pattern = os.path.join(path_to_json,'*_UTC.json')
file_list = glob.glob(json_pattern)

for file in json_files:
    # print(file)
    data = pd.read_json(file, orient=str)
    postnode = data['node']
    temp = temp.append(postnode, ignore_index = True)

In [34]:
temp.shape

(1248, 50)

In [35]:
temp.head(3)

,__typename,accessibility_caption,caption_is_edited,commenting_disabled_for_viewer,comments_disabled,dash_info,dimensions,display_resources,display_url,edge_liked_by,...,version,video_duration,video_url,video_view_count,viewer_can_reshare,viewer_has_liked,viewer_has_saved,viewer_has_saved_to_collection,viewer_in_photo_of_you,edge_sidecar_to_children
0,GraphVideo,None,1.0,0.0,0.0,"{'is_dash_eligible': True, 'number_of_qualitie...","{'height': 937, 'width': 750}","[{'config_height': 800, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 234},...,NaN,18.833,https://scontent-sjc3-1.cdninstagram.com/v/t50...,2740.0,1.0,0.0,0.0,0.0,0.0,NaN
1,GraphImage,Photo by Kathy Wingo in Ralph Brennan's Jazz K...,0.0,0.0,0.0,NaN,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 12},...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,NaN
2,GraphImage,Photo by 𝗘𝗻𝗴𝗹𝗶𝘀𝗵𝗺𝗮𝗻 & 𝗰𝗼𝗿𝗴𝗶 in FIGat7th. Image...,0.0,0.0,0.0,NaN,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 33},...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,NaN


In [57]:
json_fnames = [f.split('.json')[0] for f in json_files]
json_fnamesS = pd.Series(json_fnames)
temp['file_name'] = json_fnamesS

In [58]:
temp2 = temp[temp['__typename'] == "GraphImage"]
print(temp2.shape)

(850, 51)


In [59]:
# temp2.columns

In [60]:
# temp2.iloc[0]

In [61]:
df = temp2[['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by', 'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp']]

In [62]:
df.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp
1,2020-03-01_02-19-47_UTC,Photo by Kathy Wingo in Ralph Brennan's Jazz K...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 12},2254934715736168674,0.0,"{'has_public_page': True, 'id': 1962700, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583029e+09
2,2020-03-01_02-58-19_UTC,Photo by 𝗘𝗻𝗴𝗹𝗶𝘀𝗵𝗺𝗮𝗻 & 𝗰𝗼𝗿𝗴𝗶 in FIGat7th. Image...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 33},2254954116237586018,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583031e+09
3,2020-03-01_03-17-11_UTC,Photo by fernandaSR. in Yard House. Image may ...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 8},2254963604793427950,0.0,"{'has_public_page': True, 'id': 16623381005376...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583033e+09
5,2020-03-01_04-47-43_UTC,Photo by Scott Olson in FIGat7th. Image may co...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 14},2255009173700278683,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583038e+09
8,2020-03-01_05-44-24_UTC,Photo by Mary Valdemar in Ralph Brennan's Jazz...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 30},2255037702045019382,0.0,"{'has_public_page': True, 'id': 1962700, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583041e+09


In [63]:
df.to_csv("post_jsons_df.csv")

In [6]:
# load progress
os.chdir(r'C:\Users\erica\OneDrive - University of Southern California\Projects_at_USC\CKIDS_current\data')
df = pd.read_csv(r"post_jsons_df.csv", index_col=0)

In [40]:
# Read in text descrip files (hashtags)

In [19]:
def read_txt(file):
    text = open(file, "r", encoding="utf8").read()
    return text   # return as string


def get_ht(text):
    ht = re.findall(r"#(\w+)", text)
    return ht

In [17]:
texts = []

for f in txt_files:
    text = open(f, 'r', encoding="utf8").read()
    texts.append(text) 

In [20]:
texts = list(map(read_txt, txt_files))
hashtags = list(map(get_ht, texts))

In [24]:
# get file names
txt_fnames = [f.split('.txt')[0] for f in txt_files]

# convert all to series, and concat them to a df
textsS = pd.Series(texts)
hashtagsS = pd.Series(hashtags)
txt_fnamesS = pd.Series(txt_fnames)

df_txt = pd.DataFrame(pd.concat([txt_fnamesS, textsS, hashtagsS], axis=1))
df_txt.columns = ['file_name', 'descrip', 'hashtags']

In [27]:
df_txt.to_csv("df_txt.csv")

In [ ]:
# Join the json df and txt df by their common filename

In [64]:
print(df_txt.columns)
print(df.columns)

Index(['file_name', 'descrip', 'hashtags'], dtype='object')
Index(['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by',
       'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp'],
      dtype='object')


In [65]:
df_txt['file_name'].head()

0    2020-03-01_00-43-27_UTC
1    2020-03-01_02-19-47_UTC
2    2020-03-01_02-58-19_UTC
3    2020-03-01_03-17-11_UTC
4    2020-03-01_03-28-06_UTC
Name: file_name, dtype: object

In [66]:
df['file_name'].head

<bound method NDFrame.head of 1       2020-03-01_02-19-47_UTC
2       2020-03-01_02-58-19_UTC
3       2020-03-01_03-17-11_UTC
5       2020-03-01_04-47-43_UTC
8       2020-03-01_05-44-24_UTC
                 ...           
1243    2020-03-26_22-29-37_UTC
1244    2020-03-26_22-53-52_UTC
1245    2020-03-26_23-04-48_UTC
1246    2020-03-26_23-07-16_UTC
1247    2020-03-26_23-45-12_UTC
Name: file_name, Length: 850, dtype: object>

In [67]:
# merge the post metadata (jsons) df and text descrip df (txt)
merged = pd.merge(df, df_txt, on='file_name')
print(merged.shape)

(801, 11)


In [68]:
merged.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp,descrip,hashtags
0,2020-03-01_02-19-47_UTC,Photo by Kathy Wingo in Ralph Brennan's Jazz K...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 12},2254934715736168674,0.0,"{'has_public_page': True, 'id': 1962700, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583029e+09,Cheers!\n,[]
1,2020-03-01_02-58-19_UTC,Photo by 𝗘𝗻𝗴𝗹𝗶𝘀𝗵𝗺𝗮𝗻 & 𝗰𝗼𝗿𝗴𝗶 in FIGat7th. Image...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 33},2254954116237586018,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583031e+09,Green grass \n#Corgi #CorgiAddict #CorgiCommun...,"[Corgi, CorgiAddict, CorgiCommunity, コーギー, 코기,..."
2,2020-03-01_03-17-11_UTC,Photo by fernandaSR. in Yard House. Image may ...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 8},2254963604793427950,0.0,"{'has_public_page': True, 'id': 16623381005376...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583033e+09,Relax time\n,[]
3,2020-03-01_04-47-43_UTC,Photo by Scott Olson in FIGat7th. Image may co...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 14},2255009173700278683,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583038e+09,Somebody call the LAFD! 🔥\n,[]
4,2020-03-01_05-44-24_UTC,Photo by Mary Valdemar in Ralph Brennan's Jazz...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 30},2255037702045019382,0.0,"{'has_public_page': True, 'id': 1962700, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583041e+09,#GrayHair <3 #BluesNJazz\n,"[GrayHair, BluesNJazz]"


In [69]:
merged.to_csv("posts_merged_df_1.csv")